<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor any WML online deployment as a self managed subscription on Openscale. For self managed subscriptions, all the monitors are run as jobs on user provided spark engine. Use the notebook to enable quality, drift, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. Configuration Archive generated using common configuration notebook [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb) and pre created WML deployment.
2. Payload, feedback, drifted transactions, explanations queue and result tables to be created or validated in an IBM DB2 storage. Please note, some of these maybe optional depending on your use-case.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Fairness monitoring](#fairness)
* [7. Explainability monitoring](#explainability)

## Setup <a name="setup"></a>

### Installing Required Libraries

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

### Import configuration archive/package

Configuration archive/package created using configuration notebook will be required to onboard model for monitoring in IBM Watson OpenScale. Provide path location of archive here.

Please note if you are executing this notebook in IBM Watson Studio, first upload the configuration archive/package to project and use provided code snippet to download it to local directory of this notebook.

In [1]:
import warnings
import tarfile
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

# Note: Restart kernel after the dependencies are installed
!pip install --upgrade ibm-watson-openscale
!pip install "ibm_wos_utils~=5.0.0"

# # Download "configuration_archive.tar.gz" from project to local directory and 
# extract the artifacts
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("configuration_archive.tar.gz")
archive_file_path = "configuration_archive.tar.gz"
with tarfile.open('configuration_archive.tar.gz', "r:gz") as tar:
    tar.extractall('.')
# On unpacking the configuration archive, the files [common_configuration.json, drift_archive.tar.gz, 
# explainability.tar.gz, fairness_statistics.json] will be extracted to the current directory. 
# Output may differ depending on which monitors were enabled while running the 
# common configuration notebook

In [2]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.39
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [3]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

WML_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [4]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [5]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [6]:
spark_parameters = {
    "max_num_executors": 1,
    "min_num_executors": 1,
    "executor_cores": 1,
    "executor_memory": 1,
    "driver_cores": 1,
    "driver_memory": 1
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.

In [7]:
JDBC_CONNECTION_NAME = ""
JDBC_CONNECTION_DESCRIPTION = ""

JDBC_HOST = ""
JDBC_PORT = ""
JDBC_USE_SSL = ""
JDBC_SSL_CERTIFICATE = ""
JDBC_DRIVER = ""
JDBC_USERNAME = ""
JDBC_PASSWORD = ""
JDBC_DATABASE_NAME = ""

JDBC_CONNECTION_NAME = "jdbc for batch parity"
JDBC_CONNECTION_DESCRIPTION = "jdbc connection from SDK"

In [8]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

2 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [9]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [10]:
#feedback

FEEDBACK_DATABASE_NAME = None
FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [11]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_SCHEMA_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [12]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [13]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [14]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [15]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [16]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WML_CREDENTIALS,
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider d7ad1164-a387-462d-b495-dd25d6241404 






active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




00000000-0000-0000-0000-000000000000,active,WML_IAE3,watson_machine_learning,2024-06-30 11:22:00.507000+00:00,d7ad1164-a387-462d-b495-dd25d6241404
None,active,OpenScale Headless Service Provider,custom_machine_learning,2024-06-29 03:21:50.385000+00:00,e67332c4-0f88-4a8f-9c41-396d28c07448
None,active,IAE3,custom_machine_learning,2024-06-28 05:11:53.191000+00:00,8ac485c9-66c8-4a9c-9786-57d2940f26e8
None,active,WML_IAE2,custom_machine_learning,2024-06-27 13:36:27.009000+00:00,8b65752d-0dc9-4715-a2a2-96ee19fb7ded
99999999-9999-9999-9999-999999999999,active,shreya-space,watson_machine_learning,2024-06-26 04:34:04.685000+00:00,34668a1a-ee92-4328-8361-05ba1c3f72bf
None,active,WML_IAE,custom_machine_learning,2024-06-23 10:16:49.805000+00:00,a6780fe7-b9aa-4b91-bf87-4ade19fd41ae
99999999-9999-9999-9999-999999999999,active,shreya,watson_machine_learning,2024-06-20 13:22:29.550000+00:00,db6e279f-6d0b-49db-a94f-4b14a12aaddc
00000000-0000-0000-0000-000000000000,active,wml_aie,watson_machine_learning,2024-06-14 06:11:43.041000+00:00,15921037-330f-479a-9b19-2e2285553bf0
00000000-0000-0000-0000-000000000000,active,wml_iae_jdbc,watson_machine_learning,2024-06-14 05:57:05.755000+00:00,33e21c9c-ae94-438a-b322-8827f221b5fe
None,active,SDK_BATCH_SUB,custom_machine_learning,2024-06-14 05:37:53.874000+00:00,210bf9fa-bdb5-4cb8-8bb5-b2f938c1d5dc


Note: First 10 records were displayed.


In [17]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "d7ad1164-a387-462d-b495-dd25d6241404",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:d7ad1164-a387-462d-b495-dd25d6241404",
    "url": "/v2/service_providers/d7ad1164-a387-462d-b495-dd25d6241404",
    "created_at": "2024-06-30T11:22:00.507000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAE3",
    "service_type": "watson_machine_learning",
    "instance_id": "00000000-0000-0000-0000-000000000000",
    "credentials": {
      "secret_id": "8d7fb178-6291-4113-b7a7-d78739476dd3"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and JDBC Storage

In [18]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system WML_IAEKB_Spark
Deleting integrated system WML_IAEKB_Spark


Deleting integrated system jdbc for batch parity


#### Spark Engine

In [19]:
spark_integrated_system = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_integrated_system_id = spark_integrated_system.metadata.id
print(spark_integrated_system)

{
  "metadata": {
    "id": "5d3c74e8-dd07-4d6e-9fe4-a216800f1de1",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:5d3c74e8-dd07-4d6e-9fe4-a216800f1de1",
    "url": "/v2/integrated_systems/5d3c74e8-dd07-4d6e-9fe4-a216800f1de1",
    "created_at": "2024-06-30T11:22:08.143000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAEKB_Spark",
    "type": "spark",
    "description": "WML_IAEKB_Spark",
    "credentials": {
      "secret_id": "9492041d-0870-4071-bacc-c0c3d8f56ad3"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/v4/analytics_engines/7d2a4875-563e-42d0-994f-518ebf6e1e42/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### JDBC Storage

In [20]:
jdbc_url = "jdbc:db2://{}:{}/{}".format(JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_integrated_system = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_integrated_system_id=jdbc_integrated_system.metadata.id
print(jdbc_integrated_system)

{
  "metadata": {
    "id": "a57bf155-06dc-4154-9fbc-f66094c6fb97",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:a57bf155-06dc-4154-9fbc-f66094c6fb97",
    "url": "/v2/integrated_systems/a57bf155-06dc-4154-9fbc-f66094c6fb97",
    "created_at": "2024-06-30T11:22:09.096000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "jdbc for batch parity",
    "type": "jdbc",
    "description": "jdbc connection from SDK",
    "credentials": {
      "secret_id": "2eab8c00-3184-4d8f-ab9c-8b7f5cdb54aa"
    },
    "connection": {
      "certificate": "",
      "db_driver": "com.ibm.db2.jcc.DB2Driver",
      "location_type": "jdbc",
      "use_ssl": true
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [21]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

7b4ca85f-6a57-447c-8101-3a5e03e436e1,model,[asset] Multi-Output Regression Headless Subscription,00000000-0000-0000-0000-000000000000,7b4ca85f-6a57-447c-8101-3a5e03e436e1,deployment_[asset] Multi-Output Regression Headless Subscription,e67332c4-0f88-4a8f-9c41-396d28c07448,active,2024-06-29 03:22:11.360000+00:00,cb83c8bf-9e3b-42bf-9447-c4fb2706b56e
c89859aa-630e-4ac7-8394-b82139e06968,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,0c86658f-d40f-4c8e-9683-bf40808c8cc9,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,active,2024-06-28 05:22:29.196000+00:00,1f1c2f8b-99ef-4bbe-a2d3-06d098091fcb
94472b5e-6658-4169-8fb3-664ab3101efb,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,133d028a-826d-43e2-ac90-c040551f8f01,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,active,2024-06-28 05:17:56.016000+00:00,3bf32c6c-bdc2-4dc0-831c-df288edae02b
1a6421f0-94d7-466d-bcbe-14931bb4a7e9,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,45490a50-f7e1-42e0-ac18-06cf8faae5af,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,error,2024-06-28 05:12:09.112000+00:00,ca5936f2-9fd1-4876-9f0c-2b43ca5b8e7f
32108fcd-4407-4435-b057-b7043e2eb6bd,model,WML_IAE2,00000000-0000-0000-0000-000000000000,ee50f3df-0c4d-4d50-b17c-09c123b5252e,WML_IAE2,8b65752d-0dc9-4715-a2a2-96ee19fb7ded,active,2024-06-27 13:38:42.493000+00:00,0f4c5577-ac33-4f23-820e-0a718b0a55c7
d4a05e9b-6f76-43f3-8045-9276d97bc9cd,model,My SDK Batch Subscription-DB2,00000000-0000-0000-0000-000000000000,9bb3d394-3b1f-4ef4-a2d9-5bff6c9ecc00,My SDK Batch Subscription-DB2,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-06-26 07:45:02.879000+00:00,9db7eff3-d92f-462c-85c0-dcfb4f7725a0
9a70ffbf-8bd7-4e6d-8520-1975832f6d31,model,shap438 - P10 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,032538a7-4549-4f87-b3fc-ed72a5888849,438features,34668a1a-ee92-4328-8361-05ba1c3f72bf,active,2024-06-26 04:34:18.520000+00:00,3fc64ae7-b852-4951-bc65-4e450b473952
3ed9f2ab-665e-48e2-8c7a-58ac3583d33d,model,My SDK Batch Subscription-DB2,00000000-0000-0000-0000-000000000000,47313ae2-8dff-4ced-8c69-aa61f2a9a70b,My SDK Batch Subscription-DB2,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-06-25 15:35:43.781000+00:00,947e3b52-efff-4b8c-bc75-9397fd682516
e307dda2-8653-4201-a2b3-aa7e89b6692b,model,My SDK Batch Subscription-DB2,00000000-0000-0000-0000-000000000000,6842d6e9-1aa6-4cd3-b0cd-185cea193b6e,My SDK Batch Subscription-DB2,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-06-25 15:28:01.182000+00:00,d46696c9-ef7c-4159-887c-4bc3718d6e73
1995cae7-3390-4dad-a0bc-5d056fe3397f,model,WML_IAE,00000000-0000-0000-0000-000000000000,88e7e864-328f-4b3f-a683-5de9849d93e2,WML_IAE,a6780fe7-b9aa-4b91-bf87-4ade19fd41ae,active,2024-06-23 10:18:00.421000+00:00,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4


Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, we expect you extracted Configuration Archive to current directory and it contains common_configuration.json. If not true, please update path to correct location. After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [22]:
import uuid
import json
# Provide the WML deployment io and space d
deployment_uid = ""
space_id = ""

common_configuration = None
with open("common_configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        raise Exception("Please provide the correct path to the common configuration JSON")
        
asset_deployment_details = wos_client.service_providers.list_assets(
    data_mart_id=data_mart_id, service_provider_id=service_provider_id, 
    deployment_id = deployment_uid, deployment_space_id = space_id).result['resources'][0]

model_asset_details_from_deployment = wos_client.service_providers.get_deployment_asset(
    data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,
    deployment_space_id=space_id)

scoring_endpoint = ScoringEndpointRequest(
    url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url'] )

    
# Set asset details
asset = Asset(
    asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
    name=model_asset_details_from_deployment["entity"]["asset"]["name"],
    url=model_asset_details_from_deployment["entity"]["asset"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=common_configuration.get("problem_type") if common_configuration.get("problem_type") else common_configuration.get("model_type")
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=asset_deployment_details['metadata']['guid'],
    name=asset_deployment_details['entity']['name'],
    deployment_type= DeploymentTypes.ONLINE,
    url=asset_deployment_details['entity']['scoring_endpoint']['url'],
    scoring_endpoint=scoring_endpoint
)

probability_fields = common_configuration.get("probability_fields") if common_configuration.get("probability_fields") else [common_configuration["probability"]]

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=probability_fields, # comment out this line for regression models as probability_fields is not applicable
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

In [23]:
# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_integrated_system_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_SCHEMA_NAME,
        table_name=FEEDBACK_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_SCHEMA_NAME,
        table_name=PAYLOAD_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_SCHEMA_NAME,
        table_name=DRIFT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:
    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME, 
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [24]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    scoring_endpoint=scoring_endpoint).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:7585a078-c1b6-4a77-9ab3-87be9fb6026c",
    "url": "/v2/subscriptions/7585a078-c1b6-4a77-9ab3-87be9fb6026c",
    "created_at": "2024-06-30T12:02:35.155000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "4d2f2fb2-6b64-4d58-8f13-257166e468e9",
    "asset": {
      "asset_id": "592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9",
      "url": "https://internal-nginx-svc:12443/ml/v4/models/592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9?space_id=088c142e-f35e-4e48-a30c-ad55a6edeecc&version=2020-06-12",
      "name": "gcr - P2 XGB Classifier - Model",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
   

In [25]:
import time
# Checking subscription status

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

active


In [26]:
# Add training, output, and input data schemas to the subscription
schemas_patch_document = [
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/training_data_schema', value=common_configuration["training_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/input_data_schema', value=common_configuration["input_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/output_data_schema', value=common_configuration["output_data_schema"])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=schemas_patch_document)

# Add data_sources to the subscription
data_sources_patch_document=[
    JsonPatchOperation(op=OperationTypes.ADD, path='/data_sources', value=[data_source.to_dict() for data_source in data_sources])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=data_sources_patch_document)

In [27]:
import time

# Checking subscription status after the patch operation

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state

while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [28]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 1000
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "url": "/v2/monitor_instances/aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "created_at": "2024-06-30T12:07:05.617000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [29]:
from datetime import datetime

quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

12:07:11 active


In [30]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "url": "/v2/monitor_instances/aa1d45d9-16c8-48cc-866c-91d12f7b47d6",
    "created_at": "2024-06-30T12:07:05.617000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-06-30T12:07:06.048000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

Please make sure you have data in feedback table in db2. Quality evaluation will use this data for its metrics computation.

In [31]:
# Trigger on-demand run
monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "06599341-cea0-40b9-9c81-edca6a999bb5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:06599341-cea0-40b9-9c81-edca6a999bb5",
    "url": "/v2/monitor_instances/30171a44-7a4e-4ebc-afda-3d0de5625cf5/runs/06599341-cea0-40b9-9c81-edca6a999bb5",
    "created_at": "2024-07-05T12:57:03.460000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "last_processed_time": "2024-07-05T09:02:21.005321Z",
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2024-07-05T12:57:03.450000Z",
      "started_at": "2024-07-05T12:57:03.460000Z",
      "operators": []
    }
  }
}


In [32]:
# Check run status
quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

13:01:09 running
13:01:39 running
13:02:09 finished


12:33:29 running


{
  "parameters": {
    "job_run_schedule_id": "3628b291-dc78-42c5-8c32-0771b60f45c6",
    "min_feedback_data_size": 10
  },
  "status": {
    "state": "error",
    "queued_at": "2024-06-30T12:28:56.363000Z",
    "started_at": "2024-06-30T12:28:56.891000Z",
    "updated_at": "2024-06-30T12:33:58.329000Z",
    "completed_at": "2024-06-30T12:33:58.274000Z",
    "message": "Quality Run Execution Failed. Error: Timestamp values are required in `scoring_timestamp` column to compute the quality metrics",
    "failure": {
      "trace": "3ae036a9-d87e-4148-8c58-78138eff8d7c",
      "errors": [
        {
          "code": "AIQFS0322E",
          "message": "Quality Run Execution Failed. Error: Timestamp values are required in `scoring_timestamp` column to compute the quality metrics"
        }
      ]
    },
    "operators": [
      {
        "id": "original",
        "status": {
          "state": "error",
          "completed_at": "2024-06-30T12:33:58.274000Z",
          "failure": {
       

### Display quality metrics

In [33]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-07-05 13:02:06.991255+00:00,true_positive_rate,597a84d1-9d73-410a-9ccb-16ea345f5082,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,area_under_roc,597a84d1-9d73-410a-9ccb-16ea345f5082,0.5952380952380952,0.8,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,precision,597a84d1-9d73-410a-9ccb-16ea345f5082,0.5,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,f1_measure,597a84d1-9d73-410a-9ccb-16ea345f5082,0.4,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,accuracy,597a84d1-9d73-410a-9ccb-16ea345f5082,0.7,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,log_loss,597a84d1-9d73-410a-9ccb-16ea345f5082,0.715401570634091,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,false_positive_rate,597a84d1-9d73-410a-9ccb-16ea345f5082,0.14285714285714285,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,area_under_pr,597a84d1-9d73-410a-9ccb-16ea345f5082,0.43333333333333335,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 13:02:06.991255+00:00,recall,597a84d1-9d73-410a-9ccb-16ea345f5082,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,06599341-cea0-40b9-9c81-edca6a999bb5,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 09:02:20.538843+00:00,true_positive_rate,521a85f5-6dd2-4483-a4fc-f3afac8da92c,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,1278a1bb-d471-482a-96c7-f458b03130a9,subscription,3046b546-e422-4737-9123-a26e025d3cef


Note: First 10 records were displayed.


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [34]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': '81d2a92f-b601-45d9-bd09-134c1638e37b',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_dis

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of minimum records required in an evaluation time window for drift analysis to be completed. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to learn drift artefacts (drift model training and data constraints learning) by IBM Watson OpenScale via Spark job accessing training data table.

In [35]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 1000,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "url": "/v2/monitor_instances/8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "created_at": "2024-07-01T05:53:36.578000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 10,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_dr

### Check monitor instance status

In [36]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_status)

06:00:37 preparing
06:01:07 preparing
06:02:37 active


### Run an on-demand evaluation
Please make sure you have data in payload table in db2. Drift evaluation will use this data for its metrics computation.

In [37]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "url": "/v2/monitor_instances/8fe6da36-b580-47d6-ae38-27e3f6b06ead",
    "created_at": "2024-07-01T05:53:36.578000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-01T06:01:03.398000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [38]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "eab7c18e-47c0-47c6-ae59-9f90c4cf1293",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:eab7c18e-47c0-47c6-ae59-9f90c4cf1293",
    "url": "/v2/monitor_instances/8fe6da36-b580-47d6-ae38-27e3f6b06ead/runs/eab7c18e-47c0-47c6-ae59-9f90c4cf1293",
    "created_at": "2024-07-01T06:31:59.443000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "last_drift_run_end_timestamp": "2024-07-01T06:30:36.347922Z",
      "last_drift_run_start_timestamp": "2024-07-01T06:28:32.206559Z",
      "last_record_window_end_timestamp": "2024-07-01T06:28:33.637175Z",
      "last_record_window_start_timestamp": "2024-07-01T06

In [39]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), drift_run_status)

06:31:59 running
06:32:29 running
06:32:59 running
06:33:30 running
06:34:00 running
{
  "parameters": {
    "batch_job_id": "e6f4f2a2-bd88-4de7-959a-f2dd41914d1c",
    "batch_job_output_path": "drift_evaluation/a8bfa541-4962-4224-b805-4a3324c23d90/output/eab7c18e-47c0-47c6-ae59-9f90c4cf1293",
    "batch_job_schedule_id": "1a2114d9-9b91-4463-89c7-91e67354fc5e",
    "batch_records_end_time": "2024-07-01T06:32:01.226594Z",
    "batch_records_start_time": "2024-07-01T06:28:33.637175Z",
    "config_status": {
      "model_name": null,
      "state": "finished"
    },
    "data_drift_enabled": true,
    "data_drift_threshold": 0.1,
    "drift_buffer_range": [
      -4.5,
      4.5
    ],
    "drift_model_version": null,
    "drift_threshold": 0.05,
    "last_drift_run_end_timestamp": "2024-07-01T06:30:36.347922Z",
    "last_drift_run_start_timestamp": "2024-07-01T06:28:32.206559Z",
    "last_record_window_end_timestamp": "2024-07-01T06:28:33.637175Z",
    "last_record_window_start_timestamp

### Display drift metrics

In [40]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-07-01 06:32:01.226594+00:00,data_drift_magnitude,bbed2417-411a-4a83-8fcd-aeab2d61a8c1,0.0,None,0.1,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,eab7c18e-47c0-47c6-ae59-9f90c4cf1293,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:32:01.226594+00:00,drift_magnitude,bbed2417-411a-4a83-8fcd-aeab2d61a8c1,0.0,None,0.05,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,eab7c18e-47c0-47c6-ae59-9f90c4cf1293,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:32:01.226594+00:00,predicted_accuracy,bbed2417-411a-4a83-8fcd-aeab2d61a8c1,0.8916666666666666,None,None,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,eab7c18e-47c0-47c6-ae59-9f90c4cf1293,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:28:33.637175+00:00,data_drift_magnitude,4981524b-5b88-4ccd-8dc0-906c4c14c77f,0.0,None,0.1,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,a14f5ef5-438f-414c-9e41-1ddae5871bd7,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:28:33.637175+00:00,drift_magnitude,4981524b-5b88-4ccd-8dc0-906c4c14c77f,0.0,None,0.05,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,a14f5ef5-438f-414c-9e41-1ddae5871bd7,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:28:33.637175+00:00,predicted_accuracy,4981524b-5b88-4ccd-8dc0-906c4c14c77f,0.8916666666666666,None,None,[],drift,8fe6da36-b580-47d6-ae38-27e3f6b06ead,a14f5ef5-438f-414c-9e41-1ddae5871bd7,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c


# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [41]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

fairness_statistics_json = None
with open("fairness_statistics.json", "r") as fp:
    fairness_statistics_json = json.load(fp)
    
parameters = fairness_statistics_json["parameters"]
thresholds = fairness_statistics_json["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "54d788f6-f9f9-4f02-9a97-d29d643fc1c3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:54d788f6-f9f9-4f02-9a97-d29d643fc1c3",
    "url": "/v2/monitor_instances/54d788f6-f9f9-4f02-9a97-d29d643fc1c3",
    "created_at": "2024-07-01T06:35:47.225000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "training_data_last_processed_time": "2024-06-30T11:07:29.827790Z",
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "majority": [
            "male"
          ],
          "t

### Check monitor instance status

In [42]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

06:35:48 preparing
06:36:18 active


### Run an on-demand evaluation
Please make sure you have data in payload table in db2. Fairness evaluation will use this data for its metrics computation.

In [43]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "86718005-e998-4729-8684-18784ab32342",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:86718005-e998-4729-8684-18784ab32342",
    "url": "/v2/monitor_instances/54d788f6-f9f9-4f02-9a97-d29d643fc1c3/runs/86718005-e998-4729-8684-18784ab32342",
    "created_at": "2024-07-01T06:36:27.581000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "minority": [
            "female"
          ],
          "threshold": 0.95
        },
        {
          "feature": "Age",
          "majority": [
            [
              26,
              75
            ]
          ],
          "metric_ids": [
            "stati

In [44]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)   
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

06:42:19 running
06:42:49 finished


### Display fairness metrics

In [45]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-07-01 06:41:31.472834+00:00,fairness_value,fbe35a3d-5bde-4e65-88dd-d0384a29dabb,175.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,54d788f6-f9f9-4f02-9a97-d29d643fc1c3,86718005-e998-4729-8684-18784ab32342,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:41:31.472834+00:00,statistical_parity_difference,fbe35a3d-5bde-4e65-88dd-d0384a29dabb,0.429,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,54d788f6-f9f9-4f02-9a97-d29d643fc1c3,86718005-e998-4729-8684-18784ab32342,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:41:31.472834+00:00,fairness_value,fbe35a3d-5bde-4e65-88dd-d0384a29dabb,142.85714285714286,95.0,125.0,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,54d788f6-f9f9-4f02-9a97-d29d643fc1c3,86718005-e998-4729-8684-18784ab32342,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c
2024-07-01 06:41:31.472834+00:00,statistical_parity_difference,fbe35a3d-5bde-4e65-88dd-d0384a29dabb,0.3,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,54d788f6-f9f9-4f02-9a97-d29d643fc1c3,86718005-e998-4729-8684-18784ab32342,subscription,7585a078-c1b6-4a77-9ab3-87be9fb6026c


# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [46]:
with open("explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [47]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation
# Lime global explanation is available from Cloud Pak for Data version 4.6.4 onwards.
#    "global_explanation": {
#        "enabled": True,  # Flag to enable global explanation 
#        "explanation_method": "lime",
#        "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
#    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "url": "/v2/monitor_instances/4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "created_at": "2024-07-01T06:54:16.277000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-01T07:08:35.333000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "config_modified_at": "2024-07-01T07:08:35.267308Z",
      "config_package_file": "explainability.tar.gz",
      "contrastive": {
        "enabled": true
      },
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
  

### Check monitor instance status

In [48]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

07:09:50 active


### Run an on-demand evaluation
Please make sure you have data in Explain queue table in db2. Explainability will use this data for computing explanations.

In [49]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "url": "/v2/monitor_instances/4ca84f62-6020-4bc0-b2b9-e30a7b05cf28",
    "created_at": "2024-07-01T06:54:16.277000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-01T07:08:35.333000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "7585a078-c1b6-4a77-9ab3-87be9fb6026c"
    },
    "parameters": {
      "config_modified_at": "2024-07-01T07:08:35.267308Z",
      "config_package_file": "explainability.tar.gz",
      "contrastive": {
        "enabled": true
      },
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
  

In [50]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "f0469bdf-b55b-49d0-88ce-9c95db5c8eb2",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:f0469bdf-b55b-49d0-88ce-9c95db5c8eb2",
    "url": "/v2/monitor_instances/4ca84f62-6020-4bc0-b2b9-e30a7b05cf28/runs/f0469bdf-b55b-49d0-88ce-9c95db5c8eb2",
    "created_at": "2024-07-05T13:02:10.983000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_modified_at": "2024-07-01T07:08:35.267308Z",
      "config_package_file": "explainability.tar.gz",
      "contrastive": {
        "enabled": true
      },
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 12
      },
      "last_run_records_end_time": "2024-07-01T07:09:12.178270Z",
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 208
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "b60f1b6e-c06f-44d9-8569

In [51]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
print(monitoring_run_details)

07:12:35 running
07:13:36 running
{
  "triggered_by": "scheduler",
  "parameters": {
    "batch_job_app_id": null,
    "batch_job_data_path": "explanations_computation_job/bad6dc83-91ae-4dc5-b018-a068f7af5894/data",
    "batch_job_id": "82920d48-7142-4616-99d5-94fe5751d982",
    "batch_job_output_path": "explanations_computation_job/bad6dc83-91ae-4dc5-b018-a068f7af5894/output/c44a6f1b-44fb-47a7-8c5d-3bb263bb9c79",
    "batch_job_poll_schedule_id": "7c9a2acb-0e33-416b-aac5-1fd1e3c816cd",
    "config_modified_at": "2024-07-01T07:08:35.267308Z",
    "config_package_file": "explainability.tar.gz",
    "contrastive": {
      "enabled": true
    },
    "controllable_features": [],
    "explanations_count": {
      "failed": 0,
      "total": 12
    },
    "lime": {
      "enabled": true,
      "features_count": 10,
      "perturbations_count": 208
    },
    "measurement_id": null,
    "records_end_time": "2024-07-01T07:09:12.178270Z",
    "records_start_time": null,
    "total_records_proce

In [ ]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
# wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [52]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result.to_dict()
print(explanations)

{'total_count': 12, 'limit': 50, 'offset': 0, 'explanation_fields': ['explanation_task_id', 'scoring_id', 'created_at', 'finished_at', 'status', 'prediction', 'subscription_id', 'deployment_id', 'asset_name', 'deployment_name', 'probability', 'explanation_type'], 'explanation_values': [['47682a56-766c-4a77-8cd1-76760c50e24a', 'sc1', '2024-07-01T07:10:33.192446Z', '2024-07-01T07:10:33.192859Z', 'finished', 'No Risk', '7585a078-c1b6-4a77-9ab3-87be9fb6026c', '2b976af0-e4ab-4859-af7d-2f2287d864ad', 'gcr - P2 XGB Classifier - Model', 'gcr model', 0.8429199, 'lime'], ['df3ebfcd-874c-4f7f-9aa2-c586a8daf5a7', 'sc2', '2024-07-01T07:10:33.200075Z', '2024-07-01T07:10:33.200286Z', 'finished', 'No Risk', '7585a078-c1b6-4a77-9ab3-87be9fb6026c', '2b976af0-e4ab-4859-af7d-2f2287d864ad', 'gcr - P2 XGB Classifier - Model', 'gcr model', 0.8291593, 'lime'], ['550f0274-65de-434f-b3b2-ff2c6a0f1150', 'sc6', '2024-07-01T07:10:33.221617Z', '2024-07-01T07:10:33.221821Z', 'finished', 'No Risk', '7585a078-c1b6-4a7

### Generate on demand explanation

Choose any record from payload logging table or Explain Queue table and generate Lime, Contrastive explanations for the record. Please note that contrastive explanations are only supported for structured classification models. So specify the explanation_types accordingly. 

In [53]:
import time
# Please enter scoring id(from payload table or Explain Queue table), for which you want to generate an explanation
# scoring_ids = ["<scoring_id_from_pl_or_explain_queue_table>"] 

# As an example, we will generate a contrastive explanation for the first scoring id from the list of explanations generated above
# as part of the Explainability monitor run
scoring_ids = [explanations["explanation_values"][0][1]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))

explanation_types = ["lime", "contrastive"]
explanation_task_ids = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, subscription_id=subscription_id, explanation_types=explanation_types).result.metadata.explanation_task_ids
explanation_task_id = explanation_task_ids[0]

print("Getting explanation for explanation task id: {}".format(explanation_task_id))
explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
while explanation.entity.status.state not in ("finished", "error"):
        explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
        explanation_run_status = explanation.entity.status.state
        if explanation_run_status not in ("finished", "error"):
            print(datetime.utcnow().strftime("%H:%M:%S"), explanation_run_status)
            time.sleep(60)
            
            
print(explanation)

Running explanations on scoring IDs: ['sc1']
Getting explanation for explanation task id: 5ca0bf01-a014-48c6-9337-66bdcdfde35a
07:17:23 in_progress
07:18:33 in_progress
{
  "metadata": {
    "explanation_task_id": "5ca0bf01-a014-48c6-9337-66bdcdfde35a",
    "created_by": "1000331001",
    "created_at": "2024-07-01T07:17:19.898366Z",
    "updated_at": "2024-07-01T07:18:31.666443Z"
  },
  "entity": {
    "status": {
      "state": "finished"
    },
    "asset": {
      "id": "592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9",
      "name": "gcr - P2 XGB Classifier - Model",
      "input_data_type": "structured",
      "problem_type": "binary",
      "deployment": {
        "id": "2b976af0-e4ab-4859-af7d-2f2287d864ad",
        "name": "gcr model"
      }
    },
    "input_features": [
      {
        "name": "CheckingStatus",
        "value": "less_0",
        "feature_type": "categorical"
      },
      {
        "name": "LoanDuration",
        "value": 18,
        "feature_type": "numerical"
      

### Cleanup the untarred files

In [54]:
import os
files = ["common_configuration.json", "explainability.tar.gz", "drift_archive.tar.gz", "fairness_statistics.json"]
for file in files:
    if os.path.isfile(file):
        os.remove(file)

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the model you configured to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.